## 朴素贝叶斯分类器（Naive Bayes)

这一部分我们探讨朴素贝叶斯分类器，大量实验证明，这个分类模型在对文本分类中能表现良好。究其原因，也许是对于邮件过滤这类任务，我们用于区分类别的文本特征彼此独立性较强，刚好模型的假设便是特征独立。 【Source Code】 

In [1]:
from sklearn.datasets import fetch_20newsgroups
#20类新闻数据，在线下载
news = fetch_20newsgroups(subset = 'all')

In [2]:
# 查验数据，依然采用dict格式，共有18846条样本
print(len(news.data),len(news.target))

18846 18846


In [3]:
news.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR', 'description'])

In [4]:
# 查验一下新闻类别和种数
print(news.target_names)
print(news.target_names.__len__())

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
20


In [5]:
len(news.filenames)

18846

In [9]:
print(news.description)

the 20 newsgroups by date dataset


In [10]:
#同样，我们选取25% 的数用来测试模型性能
from sklearn.cross_validation import train_test_split

X_train,X_test,y_train,y_test = train_test_split(news.data,news.target,test_size = 0.25)

In [12]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

14134
4712
14134
4712


### 许多原始数据无法被分类器直接使用
### 图像可以直接使用pixel信息，文本则需要进一步处理成数值化信息

In [13]:
from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import *
from scipy.stats import sem

#我们在NB_classifier的基础上，对比几种特征抽取方法的性能，并且使用Pipline简化构建训练流程
# 1.使用计数方式，返回词频信息，未归一化
clf_1 = Pipeline([('count_vec',CountVectorizer()),('mnb',MultinomialNB())])
# 2.使用哈希
clf_2 = Pipeline([('hash_vec',HashingVectorizer(non_negative=True)),('mnb',MultinomialNB())])
# 3.使用TF-IDF词频统计，经过加权后，返回归一化特征向量，与 1 类似
clf_3 = Pipeline([('tfidf_vec',TfidfVectorizer()),('mnb',MultinomialNB())])

In [14]:
# 构造一个便于交叉验证模型性能的函数（模块）
def evaluate_cross_validation(clf,X,y,K):
    # KFold 函数需要如下参数：数据量，叉验次数，是否洗牌
    cv = KFold(len(y),K,shuffle = True,random_state = 0)
    # 采用上述的分割方式进行交叉验证，测试模型性能，
    # 对于分类问题，这些得分默认是accuracy，也可以修改为其他
    scores = cross_val_score(clf,X,y,cv = cv)
    print(scores)
    print('Mean score: %.3f (+/-%.3f)' % (scores.mean(),sem(scores)))

In [15]:
clfs = [clf_1,clf_2,clf_3]
for clf in clfs:
    evaluate_cross_validation(clf,X_train,y_train,5)

[ 0.82985497  0.84152812  0.84754156  0.83728334  0.84961076]
Mean score: 0.841 (+/-0.004)


C:\Users\97855\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\feature_extraction\hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
C:\Users\97855\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\feature_extraction\hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
C:\Users\97855\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\feature_extraction\hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
C:\Users\97855\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\feature_extraction\hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in vers

[ 0.74319066  0.74000707  0.7424832   0.7591086   0.75230007]
Mean score: 0.747 (+/-0.004)
[ 0.83162363  0.83268482  0.83692961  0.84471171  0.84642604]
Mean score: 0.838 (+/-0.003)


In [17]:
# 从上述结果来看，两个特征提取的方法得到的性能相当，
# 让我们选取其中之一，进一步靠特征的精细筛选提高性能

# 添加停用词过滤，默认选择英语通用词，像像a，an，the，助动词do，be，will，介词on，around，beneath等
clf_4 = Pipeline([('tfidf_vec_adv',TfidfVectorizer(stop_words='english')),('mnb',MultinomialNB())])
evaluate_cross_validation(clf_4,X_train,y_train,5)

[ 0.86911921  0.86841174  0.86841174  0.87725504  0.87084218]
Mean score: 0.871 (+/-0.002)


In [18]:
# 如果再尝试修改贝叶斯分类器的平滑参数，也许性能会更上一层楼
clf_5 = Pipeline([('tfidf_vec_adv',TfidfVectorizer(stop_words='english')),('mnb',MultinomialNB(alpha=0.01))])
evaluate_cross_validation(clf_5,X_train,y_train,5)

[ 0.90590732  0.90696852  0.91192076  0.91262823  0.90233546]
Mean score: 0.908 (+/-0.002)


In [19]:
# 词根还原，即不同词性都算作特征
# 添加停用词过滤，默认选择英语通用词，像像a，an，the，助动词do，be，will，介词on，around，beneath等
clf_6 = Pipeline([('tfidf_vec_adv',TfidfVectorizer(binary=True,stop_words='english')),('mnb',MultinomialNB())])
evaluate_cross_validation(clf_6,X_train,y_train,5)

[ 0.85709232  0.86805801  0.87265653  0.87477892  0.86694975]
Mean score: 0.868 (+/-0.003)
